In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import math
import string
import re
from nltk.corpus import stopwords
stopw=set(stopwords.words('english'))
from nltk import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag
tweettk=TweetTokenizer()

In [2]:
links = pd.read_csv("data_process/redirected_links.csv",names=["Enlace Real"],  converters={'Enlace Real': eval})
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
text = pd.read_csv('train.csv',usecols =['text'],encoding="utf-8")

In [3]:
test['target'] = 2
totaldata = pd.concat([train,test])

In [4]:
totaldata.shape

(10876, 5)

In [5]:
train = pd.merge(links, train, right_index=True, left_index=True)

In [6]:
######################### KEYWORDS

In [7]:
tkeyword = train[train['target']==1][['id','keyword']].groupby('keyword').agg('count')
tkeyword.columns = ['keyword_t_count']
tkeyword.reset_index(inplace=True)

In [8]:
fkeyword = train[train['target']==0][['id','keyword']].groupby('keyword').agg('count')
fkeyword.columns = ['keyword_f_count']
fkeyword.reset_index(inplace=True)

In [9]:
keyword = tkeyword.merge(fkeyword)

In [10]:
keyword['keyword_metric']= keyword['keyword_t_count'] - keyword['keyword_f_count']
keyword['keyword_total']= keyword['keyword_t_count'] + keyword['keyword_f_count']
keyword['keyword_metric'] = keyword['keyword_metric']/keyword['keyword_total']
keyword = keyword.sort_values(by='keyword_total')
keyword['keyword_total'] = keyword['keyword_total'].apply(lambda x: 1 if x<20 else math.log(x+10,20))
scaler = MinMaxScaler()
x = keyword['keyword_total'].values.reshape(-1, 1)
keyword['keyword_total'] = scaler.fit_transform(x)
keyword['keyword_metric'] = keyword['keyword_metric'] * keyword['keyword_total']

In [11]:
keyword.drop('keyword_total',axis=1,inplace=True)

In [12]:
keyword['keyword_metric'].max(),keyword['keyword_metric'].min()

(0.8604936840518713, -0.8802189264555614)

In [13]:
keyword.nlargest(10,columns='keyword_metric')

,keyword,keyword_t_count,keyword_f_count,keyword_metric
150,outbreak,39,1,0.860494
202,typhoon,37,1,0.819880
149,oil%20spill,37,1,0.819880
184,suicide%20bombing,32,1,0.710830
163,rescuers,32,3,0.664208
182,suicide%20bomb,32,3,0.664208
183,suicide%20bomber,30,1,0.663827
144,nuclear%20disaster,31,3,0.641872
93,evacuated,32,4,0.640389
159,razed,31,4,0.618401


In [14]:
keyword.nsmallest(10,columns='keyword_metric')

,keyword,keyword_t_count,keyword_f_count,keyword_metric
28,body%20bags,1,40,-0.880219
167,ruin,1,36,-0.798962
120,hellfire,2,37,-0.794959
216,wrecked,3,36,-0.749533
116,harm,4,37,-0.744801
18,blazing,1,33,-0.733567
170,screaming,2,34,-0.731873
95,explode,3,35,-0.728782
7,armageddon,5,37,-0.719660
26,body%20bag,1,32,-0.710830


In [15]:
keyword=keyword[['keyword','keyword_metric']]
keyword.to_csv('data_process/keyword.csv',index=False)
totaldata = totaldata.merge(keyword, how='left')

In [16]:
totaldata.shape

(10876, 6)

In [17]:
totaldata.isna().any()

id                False
keyword            True
location           True
text              False
target            False
keyword_metric     True
dtype: bool

In [18]:
######################### DOMINIOS

In [19]:
lista = list()
def get_domain(string):
    if len(string)==0:
        return(' ')
    for s in string.split("/"):
        if "." in s:
            return s.replace("www.","")
for index,row in train.loc[train["Enlace Real"].str.len()>0].iterrows():
    for url in row["Enlace Real"]:
        lista.append([index, url, row["target"],get_domain(url)])

In [20]:
enlaces_target = pd.DataFrame(data=lista,columns=["id","Enlace","target","Dominio"])

In [21]:
enlaces_t = enlaces_target.loc[enlaces_target.target == 1]['Dominio'].value_counts()
enlaces_t = pd.DataFrame({'Dominio': enlaces_t.index, 'cant_verdaderos': enlaces_t.values})
enlaces_f = enlaces_target.loc[enlaces_target.target == 0]['Dominio'].value_counts()
enlaces_f = pd.DataFrame({'Dominio': enlaces_f.index, 'cant_falsos': enlaces_f.values})

enlaces = pd.merge(enlaces_t, enlaces_f, how='inner', on='Dominio')
enlaces.head(5)

,Dominio,cant_verdaderos,cant_falsos
0,twitter.com,481,600
1,bbc.co.uk,62,2
2,youtube.com,55,110
3,abcnews.go.com,49,5
4,facebook.com,40,51


In [22]:
enlaces['domain_metric'] = enlaces['cant_verdaderos'] - enlaces['cant_falsos']
enlaces['cant_total'] = enlaces['cant_verdaderos'] + enlaces['cant_falsos']
enlaces['domain_metric'] = enlaces['domain_metric']/enlaces['cant_total']
enlaces = enlaces.sort_values(by='cant_total')
enlaces['cant_total'] = enlaces['cant_total'].apply(lambda x: 1 if x<20 else math.log(x+10,20))
scaler = MinMaxScaler()
x = enlaces['cant_total'].values.reshape(-1, 1)
enlaces['cant_total'] = scaler.fit_transform(x)
enlaces['domain_metric'] = enlaces['domain_metric'] * enlaces['cant_total']

In [23]:
enlaces = enlaces[['Dominio','domain_metric']]
enlaces_target = enlaces_target.merge(enlaces).groupby('id').agg({'domain_metric':['mean','std','max','min']})
enlaces_target.columns =enlaces_target.columns.get_level_values(0) + '_' + enlaces_target.columns.get_level_values(1)
enlaces_target.reset_index(inplace=True)

In [24]:
enlaces_target.to_csv('data_process/enlaces.csv',index=False)

In [25]:
totaldata = totaldata.merge(enlaces_target,how='left')

In [26]:
############################# 

In [27]:
totaldata['textsize'] = totaldata['text'].apply(len) #### textsize
totaldata['punctuation_count'] = totaldata['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
totaldata['word_mean_len'] = totaldata['text'].apply(lambda x: np.mean([len(c) for c in str(x).split()])) 
totaldata['haslocacion'] = totaldata['location'].notna()
totaldata['word_count'] = totaldata['text'].apply(lambda x: len(str(x).split()))
totaldata['upper_count'] = totaldata['text'].apply(lambda x: len(re.findall("[ A-Z ]",x)))
totaldata['white_count'] = totaldata['text'].apply(lambda x: len(re.findall(r'\s+',x)))
totaldata['url_count'] = totaldata['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
totaldata['hashtag_count'] = totaldata['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
totaldata['mention_count'] = totaldata['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

In [28]:
totaldata.columns

Index(['id', 'keyword', 'location', 'text', 'target', 'keyword_metric',
       'domain_metric_mean', 'domain_metric_std', 'domain_metric_max',
       'domain_metric_min', 'textsize', 'punctuation_count', 'word_mean_len',
       'haslocacion', 'word_count', 'upper_count', 'white_count', 'url_count',
       'hashtag_count', 'mention_count'],
      dtype='object')

In [29]:
############################ TAGGING

In [30]:
tagged = pd.read_csv('data_process/textcleaned.csv')
tagged = tagged['text']
trashwords = ['https','co','http','t','s']

In [31]:
def tagging(s):
    s = s.lstrip()
    s = re.sub(r'[^a-zA-Z]', ' ',  s)
    s = re.sub(r' +', ' ',  s)
    s = tweettk.tokenize(s)
    s = [i for i in s if i not in trashwords]
    s= pos_tag(s)
    return(s)
tagged = tagged.apply(tagging)
tagged = tagged.apply(lambda x: FreqDist([i[1] for i in x]))

In [32]:
tagged = pd.DataFrame(tagged.tolist()).fillna(0)
tagged.to_csv('data_process/tagged.csv',index=False)

In [33]:
tagged

,PRP$,NNS,VBP,DT,NNP,IN,NN,PRP,VBD,TO,...,WDT,RBR,RBS,FW,PDT,POS,UH,SYM,WP$,''
0,1.0,1.0,1.0,3.0,4.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,2.0,1.0,1.0,3.0,4.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,2.0,2.0,4.0,3.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10871,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10872,0.0,1.0,1.0,0.0,4.0,5.0,3.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10873,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10874,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
#tagged = tagged[['NN','IN','RB','VBP','VB','PRP','CC','PRP$']]

In [34]:
totaldata = pd.concat([totaldata,tagged],axis=1).fillna(0)

In [35]:
totaldata.to_csv('data_process/totaldata.csv',index=False)

In [36]:
totaldata.shape

(10876, 56)